In [1]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

RANDOM_SEED = 42

housing = sklearn.datasets.fetch_california_housing()

x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=RANDOM_SEED)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(15480, 8) (15480,)
(5160, 8) (5160,)


# Baseline

In [2]:
regressor = DecisionTreeRegressor(
    random_state=RANDOM_SEED,
    min_samples_leaf=5,
    min_samples_split=20
)
regressor.fit(x_train, y_train)
baseline_acc = regressor.score(x_test, y_test)
print(baseline_acc)

0.6985753014500338


In [12]:
from decision_tree.DecisionTreeRegressor import CustomDecisionTreeRegressor

custom_regressor = CustomDecisionTreeRegressor()
custom_regressor.fit(x_train, y_train)
print(custom_regressor.score(x_test, y_test))



Node split: feature_index=0, threshold=3.72695, left_size=8507, right_size=6973
Node split: feature_index=5, threshold=3.0476465268722115, left_size=5229, right_size=3278
Node split: feature_index=6, threshold=35.489999999999995, left_size=2436, right_size=2793
Node split: feature_index=7, threshold=-118.785, left_size=280, right_size=2156
Node split: feature_index=7, threshold=-120.065, left_size=98, right_size=182
Node split: feature_index=7, threshold=-120.53, left_size=54, right_size=44
Node split: feature_index=7, threshold=-120.64, left_size=37, right_size=17
Node split: feature_index=7, threshold=-120.345, left_size=40, right_size=4
Node split: feature_index=0, threshold=1.87685, left_size=3, right_size=37
Node split: feature_index=6, threshold=35.375, left_size=138, right_size=44
Node split: feature_index=6, threshold=34.305, left_size=34, right_size=104
Node split: feature_index=6, threshold=35.355000000000004, left_size=88, right_size=16
Node split: feature_index=6, threshold

In [14]:
from decision_tree.DecisionTreeRegressor import CustomDecisionTreeRegressor

custom_regressor_max_features = CustomDecisionTreeRegressor(max_features=2)
custom_regressor_max_features.fit(x_train, y_train)
print(custom_regressor_max_features.score(x_test, y_test))

Node split: feature_index=2, threshold=6.482989625506073, left_size=13002, right_size=2478
Node split: feature_index=7, threshold=-120.09, left_size=5078, right_size=7924
Node split: feature_index=5, threshold=2.4567187198766147, left_size=1776, right_size=3302
Node split: feature_index=1, threshold=37.0, left_size=1071, right_size=705
Node split: feature_index=0, threshold=3.60515, left_size=686, right_size=385
Node split: feature_index=4, threshold=1406.5, left_size=456, right_size=230
Node split: feature_index=6, threshold=37.875, left_size=155, right_size=301
Node split: feature_index=6, threshold=37.61, left_size=89, right_size=66
Node split: feature_index=5, threshold=2.1233039997645466, left_size=54, right_size=35
Node split: feature_index=7, threshold=-121.64500000000001, left_size=43, right_size=11
Node split: feature_index=5, threshold=1.74371264642601, left_size=14, right_size=29
Node split: feature_index=2, threshold=4.80247376311844, left_size=49, right_size=17
Node split: